In [2]:
import pandas as pd

In [ ]:
df = pd.read_csv('../notion_data/Nasdaq.csv')

In [4]:
df.head()

,Unnamed: 0,Price,Close,High,Low,Open,Volume,Daily Return (%)
0,4,2024-12-26,20020.359375,20070.080078,19888.109375,19979.250000,6467910000,-0.053774
1,5,2024-12-24,20031.130859,20031.130859,19813.759766,19818.480469,4739190000,1.347086
2,6,2024-12-23,19764.880859,19772.099609,19504.679688,19641.050781,6820720000,0.982400
3,7,2024-12-20,19572.599609,19758.609375,19168.380859,19190.039062,10962450000,1.031500
4,8,2024-12-19,19372.769531,19623.019531,19357.759766,19565.660156,8029590000,-0.102719


In [5]:
df2 = df.copy()

In [8]:
df = df[['Price', 'Close']]

In [17]:
df.rename(columns={'Price' : '날짜',
                   'Close' : '원자료'}, inplace= True)

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2513 entries, 2512 to 0
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   날짜      2513 non-null   object 
 1   원자료     2513 non-null   float64
dtypes: float64(1), object(1)
memory usage: 58.9+ KB


In [19]:
df.head()

,날짜,원자료
2512,2015-01-02,4726.810059
2511,2015-01-05,4652.569824
2510,2015-01-06,4592.740234
2509,2015-01-07,4650.470215
2508,2015-01-08,4736.189941


In [20]:
df = df.sort_values('날짜').reset_index(drop=True)


In [21]:
df.head()

,날짜,원자료
0,2015-01-02,4726.810059
1,2015-01-05,4652.569824
2,2015-01-06,4592.740234
3,2015-01-07,4650.470215
4,2015-01-08,4736.189941


In [23]:
import pandas as pd

# 예시 DataFrame (여기선 df가 이미 정의되어 있다고 가정)
print("변환 전 '원자료' 컬럼 타입:")
print(df['원자료'].dtype)

print("\n변환 전 '원자료' 컬럼 내용:")
print(df['원자료'])

# 1. 문자열로 변환한 후 쉼표 제거
df['원자료'] = df['원자료'].astype(str).str.replace(',', '', regex=False)

# 2. 숫자로 변환 (변환 불가능한 값은 NaN 처리)
df['원자료'] = pd.to_numeric(df['원자료'], errors='coerce')

print("\n--- 변환 후 ---")
print("변환 후 '원자료' 컬럼 타입:")
print(df['원자료'].dtype)

print("\n변환 후 '원자료' 컬럼 내용:")
print(df['원자료'])


변환 전 '원자료' 컬럼 타입:
float64

변환 전 '원자료' 컬럼 내용:
0        4726.810059
1        4652.569824
2        4592.740234
3        4650.470215
4        4736.189941
            ...     
2508    19372.769531
2509    19572.599609
2510    19764.880859
2511    20031.130859
2512    20020.359375
Name: 원자료, Length: 2513, dtype: float64

--- 변환 후 ---
변환 후 '원자료' 컬럼 타입:
float64

변환 후 '원자료' 컬럼 내용:
0        4726.810059
1        4652.569824
2        4592.740234
3        4650.470215
4        4736.189941
            ...     
2508    19372.769531
2509    19572.599609
2510    19764.880859
2511    20031.130859
2512    20020.359375
Name: 원자료, Length: 2513, dtype: float64


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2513 entries, 0 to 2512
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   날짜      2513 non-null   object 
 1   원자료     2513 non-null   float64
dtypes: float64(1), object(1)
memory usage: 39.4+ KB


In [25]:
df['날짜'] = pd.to_datetime(df['날짜'], errors='coerce')


In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2513 entries, 0 to 2512
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   날짜      2513 non-null   datetime64[ns]
 1   원자료     2513 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 39.4 KB


In [27]:
# '날짜'를 인덱스로 설정
df = df.set_index('날짜')

# 일별 수익률 계산 (pct_change는 전일 대비 퍼센트 변동)
returns = df.pct_change()

# 결과 저장용 DataFrame 생성
result = pd.DataFrame(index=df.index)

# (shift, 라벨) 쌍 정의: 
# shift=1 → -1~-5일, shift=6 → -6~-10일, shift=11 → -11~-15일, shift=16 → -16~-20일
windows = [
    (1, '(-1~-5)'),
    (6, '(-6~-10)'),
    (11, '(-11~-15)'),
    (16, '(-16~-20)')
]

# 각 컬럼에 대해 네 구간별 표준편차 계산
for col in df.columns:
    for shift, label in windows:
        result[f"{col}{label}"] = (
            returns[col]
            .shift(shift)              # 과거 shift 일만큼 뒤로
            .rolling(window=5)         # 5일간
            .std()                     # 표준편차
        )

# 인덱스를 '날짜' 컬럼으로 되돌리고
result = result.reset_index()

# 결과 확인
print(result.head())
# 필요시 to_csv 등으로 저장



          날짜  원자료(-1~-5)  원자료(-6~-10)  원자료(-11~-15)  원자료(-16~-20)
0 2015-01-02         NaN          NaN           NaN           NaN
1 2015-01-05         NaN          NaN           NaN           NaN
2 2015-01-06         NaN          NaN           NaN           NaN
3 2015-01-07         NaN          NaN           NaN           NaN
4 2015-01-08         NaN          NaN           NaN           NaN


In [28]:
result.head(30)

,날짜,원자료(-1~-5),원자료(-6~-10),원자료(-11~-15),원자료(-16~-20)
0,2015-01-02,NaN,NaN,NaN,NaN
1,2015-01-05,NaN,NaN,NaN,NaN
2,2015-01-06,NaN,NaN,NaN,NaN
3,2015-01-07,NaN,NaN,NaN,NaN
4,2015-01-08,NaN,NaN,NaN,NaN
5,2015-01-09,NaN,NaN,NaN,NaN
6,2015-01-12,0.015427,NaN,NaN,NaN
7,2015-01-13,0.013940,NaN,NaN,NaN
8,2015-01-14,0.011919,NaN,NaN,NaN
9,2015-01-15,0.010931,NaN,NaN,NaN


In [29]:
result = result.rename(columns={
    '원자료(-1~-5)' : '나스닥수익률표준편차(-1~-5)',
    '원자료(-6~-10)' : '나스닥수익률표준편차(-6~-10)',
    '원자료(-11~-15)' : '나스닥수익률표준편차(-11~-15)',
    '원자료(-16~-20)' : '나스닥수익률표준편차(-16~-20)',
    '기타' : '기타'
})

In [30]:
macro_df = pd.read_csv('../data/macro_data.csv')

In [34]:
macro_df.columns

Index(['회사명', 'stock_code', '상장일', 'kospi200(-20)', 'nasdaq(-20)',
       'putcall(-20)', 'VIXCLS', '미국10년물국채금리수익률(-20)', '시장금리국고채3년',
       '환율변동률(-20)',
       ...
       '미국_대두(-11~-15)', '미국_대두(-16~-20)', '코스닥수익률표준편차(-1~-5)',
       '코스닥수익률표준편차(-6~-10)', '코스닥수익률표준편차(-11~-15)', '코스닥수익률표준편차(-16~-20)',
       '코스피수익률표준편차(-1~-5)', '코스피수익률표준편차(-6~-10)', '코스피수익률표준편차(-11~-15)',
       '코스피수익률표준편차(-16~-20)'],
      dtype='object', length=164)

In [31]:
# 1) Convert both columns to datetime
macro_df['상장일'] = pd.to_datetime(macro_df['상장일'])
result['날짜'] = pd.to_datetime(result['날짜'])

# 2) Sort both DataFrames by their date keys
merged_full = macro_df.sort_values('상장일').reset_index(drop=True)
result = result.sort_values('날짜').reset_index(drop=True)

merged_full = pd.merge_asof(
    merged_full,
    result,
    left_on='상장일',
    right_on='날짜',
    direction='backward'   # 상장일 당일 또는 그 이전의 가장 가까운 거래일
).drop(columns=['날짜'])

In [33]:
merged_full.columns

Index(['회사명', 'stock_code', '상장일', 'kospi200(-20)', 'nasdaq(-20)',
       'putcall(-20)', 'VIXCLS', '미국10년물국채금리수익률(-20)', '시장금리국고채3년',
       '환율변동률(-20)',
       ...
       '코스닥수익률표준편차(-11~-15)', '코스닥수익률표준편차(-16~-20)', '코스피수익률표준편차(-1~-5)',
       '코스피수익률표준편차(-6~-10)', '코스피수익률표준편차(-11~-15)', '코스피수익률표준편차(-16~-20)',
       '나스닥수익률표준편차(-1~-5)', '나스닥수익률표준편차(-6~-10)', '나스닥수익률표준편차(-11~-15)',
       '나스닥수익률표준편차(-16~-20)'],
      dtype='object', length=168)

In [35]:
merged_full.to_csv('macro_data.csv', encoding='utf-8-sig', index=False)
